# Micro-SAM Training from OMERO Data (Automated)

Train micro-SAM models using annotation tables from OMERO with automated data preparation.

## 1. Setup

In [2]:
# Import the package with training convenience functions
from omero_annotate_ai import (
    create_omero_connection_widget,
    create_training_data_widget,
    prepare_training_data_from_table,
    setup_training,    # Convenience function for training setup
    run_training       # Convenience function for training execution
)

# Additional imports
import datetime
from pathlib import Path
import torch

print(f"Available functions: Connection, Training Data, Training Utils")
print(f"Device available: {'CUDA' if torch.cuda.is_available() else 'CPU'}")


Available functions: Connection, Training Data, Training Utils
Device available: CPU


## 2. OMERO Connection

In [ ]:
# Create and display OMERO connection widget
conn_widget = create_omero_connection_widget()
conn_widget.display()

📄 Loaded configuration from connection history: root@localhost
🔐 Password loaded from keychain (no expiration)


In [5]:
# Get the OMERO connection
conn = conn_widget.get_connection()

if conn is None:
    raise ConnectionError("No OMERO connection established.")

print(f"Connected to OMERO as: {conn.getUser().getName()}")

Connected to OMERO as: root


## 3. Training Data Selection

In [6]:
# Create training data selection widget
training_widget = create_training_data_widget(connection=conn)
training_widget.display()

In [7]:
# Get selected training table
selected_table_id = training_widget.get_selected_table_id()
selected_table_info = training_widget.get_selected_table_info()

if selected_table_id:
    print(f"Selected training table:")
    print(f"  Table ID: {selected_table_id}")
    print(f"  Table Name: {selected_table_info.get('name', 'Unknown')}")
    print(f"  Created: {selected_table_info.get('created', 'Unknown')}")
else:
    raise ValueError("No training table selected. Please select a table above.")

Selected training table:
  Table ID: 1373
  Table Name: micro_sam_training_micro_sam_foci_test_20250823_105428
  Created: Unknown


## 4. Setup Training Directory

In [8]:
# Create output directory for training
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
home_dir = Path.home()
models_dir = home_dir / "micro-sam_models"
models_dir.mkdir(exist_ok=True)

folder_name = f"micro-sam-{timestamp}"
output_directory = models_dir / folder_name
output_directory.mkdir(exist_ok=True)

print(f"Training output directory: {output_directory}")

Training output directory: C:\Users\Maarten\micro-sam_models\micro-sam-20250824_111457


## 5. Automated Data Preparation

Use the automated data preparation function to download and organize training data.

In [9]:
# Run automated data preparation
try:
    training_result = prepare_training_data_from_table(
        conn=conn,
        table_id=selected_table_id,
        output_dir=output_directory,
        validation_split=0.2,  # 20% for validation
        clean_existing=True
    )
    
    print("\nTraining data preparation completed successfully!")
    print(f"\nDataset statistics:")
    for key, value in training_result['stats'].items():
        print(f"  {key}: {value}")
    
    # Store paths for later use in training
    training_input_dir = training_result['training_input']
    training_label_dir = training_result['training_label']
    val_input_dir = training_result['val_input']
    val_label_dir = training_result['val_label']
    
    print(f"\nDirectory structure created:")
    print(f"  Training images: {training_input_dir}")
    print(f"  Training labels: {training_label_dir}")
    print(f"  Validation images: {val_input_dir}")
    print(f"  Validation labels: {val_label_dir}")
    
except Exception as e:
    print(f"Error during data preparation: {e}")
    raise

Loaded table with 3 rows
Table schema validated for processing
Using 2 training images and 1 validation images


Preparing training data:   0%|          | 0/2 [00:00<?, ?it/s]INFO:omero.gateway:Registered 480f3c2e-5dd6-4977-b35c-94b060bb1e30/45d223fb-33aa-4f46-82f6-7642b604272fomero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 34317 -t 60000
INFO:omero.gateway:Unregistered 480f3c2e-5dd6-4977-b35c-94b060bb1e30/45d223fb-33aa-4f46-82f6-7642b604272fomero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 34317 -t 60000
Preparing training data:  50%|█████     | 1/2 [00:03<00:03,  3.97s/it]INFO:omero.gateway:Registered 480f3c2e-5dd6-4977-b35c-94b060bb1e30/ea9621dd-c63f-4ff5-82c2-d99880270c58omero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 34317 -t 60000
INFO:omero.gateway:Unregistered 480f3c2e-5dd6-4977-b35c-94b060bb1e30/ea9621dd-c63f-4ff5-82c2-d99880270c58omero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 34317 -t 60000
Preparing val data:   0%|          | 0/1 [00:00<?, ?it/s]INFO:omero.gateway:Registered 480f3c2e-5dd6-4977-b35c-94b060bb1e30/abf0956d-6f8c-46ea-aede-0211781f4fd3omero.ap

✅ Training data prepared successfully in: C:\Users\Maarten\micro-sam_models\micro-sam-20250824_111457
Statistics: {'n_training_images': 2, 'n_training_labels': 2, 'n_val_images': 1, 'n_val_labels': 1, 'total_rows_processed': 3}

Training data preparation completed successfully!

Dataset statistics:
  n_training_images: 2
  n_training_labels: 2
  n_val_images: 1
  n_val_labels: 1
  total_rows_processed: 3

Directory structure created:
  Training images: C:\Users\Maarten\micro-sam_models\micro-sam-20250824_111457\training_input
  Training labels: C:\Users\Maarten\micro-sam_models\micro-sam-20250824_111457\training_label
  Validation images: C:\Users\Maarten\micro-sam_models\micro-sam-20250824_111457\val_input
  Validation labels: C:\Users\Maarten\micro-sam_models\micro-sam-20250824_111457\val_label


## 6. Micro-SAM Training Setup

Configure and run micro-SAM training using the prepared data.

In [12]:
# ✨ Setup training configuration using convenience function
training_config = setup_training(
    training_result,
    epochs=10,               # Primary parameter: number of epochs (use 50+ for real training)
    batch_size=1,            # Adjust based on GPU memory
    learning_rate=1e-5,      # Conservative learning rate
    patch_shape=(512, 512),  # Input patch size
    model_type="vit_b",       # SAM model variant
    n_objects_per_batch=25   # Objects per batch for sampling
)

print("Training configuration prepared!")
print(f'Model name: {training_config["model_name"]}')
print(f'Output directory: {training_config["output_dir"]}')
print(f'Training epochs: {training_config["epochs"]}')
print(f'Calculated iterations: {training_config["n_iterations"]}')


Training configuration prepared!
Model name: micro_sam_training_20250824_111602
Output directory: C:\Users\Maarten\micro-sam_models\micro-sam-20250824_111457
Training epochs: 10
Calculated iterations: 20


In [14]:
# ✨ Execute training with convenience function
print("Starting micro-SAM training...")

training_results = run_training(training_config, framework="microsam")

print(f'🎉 Training completed successfully!')
print(f'Training Results:')
print(f'  Model name: {training_results["model_name"]}')
print(f'  Final model: {training_results.get("final_model_path", "Not available")}')
print(f'  Checkpoints saved: {len(training_results.get("checkpoints", []))}')
print(f'  Output directory: {training_results["output_dir"]}')


Starting micro-SAM training...
Starting micro-SAM training...
Model name: micro_sam_training_20250824_111602
Model type: vit_b
Training configuration:
  Patch shape: (512, 512)
  Batch size: 1
  Learning rate: 1e-05
  Iterations: 20
  Objects per batch: 25
  Checkpoint folder: C:\Users\Maarten\micro-sam_models\micro-sam-20250824_111457\checkpoints


AttributeError: module 'micro_sam.training' has no attribute 'get_sam_loader'

## 7. Model Export and Summary

In [ ]:
# Find the best checkpoint
checkpoints = list(checkpoint_folder.glob("*.pt"))
if checkpoints:
    latest_checkpoint = sorted(checkpoints)[-1]
    print(f"Latest checkpoint: {latest_checkpoint}")
    
    # Export model for inference
    export_path = output_directory / f"{model_name}_final.pt"
    print(f"Model exported to: {export_path}")
else:
    print("No checkpoints found.")

print(f"\nTraining summary:")
print(f"  Output directory: {output_directory}")
print(f"  Model name: {model_name}")
print(f"  Training completed with {n_iterations} iterations")
print(f"  Dataset statistics: {training_result['stats']}")

## 8. Cleanup

In [ ]:
# Close OMERO connection
if conn is not None:
    conn.close()
    print("OMERO connection closed.")
else:
    print("No active OMERO connection to close.")